In [159]:

import sys
sys.dont_write_bytecode = True
import pandas as pd
%load_ext autoreload
%autoreload 2
import os
os.chdir('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
from mlops.tracking.mlflow import *

In [160]:
exp = mlflow.search_experiments()[0]

In [148]:
exp.name

'Ridge'

In [161]:
check_registry_if_not_exist(exp)

/home/atheer/Desktop/mlops-project/notebooks/../mlops/tracking/mlflow.py:79: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(model_name, stages=["None"])[0]
Registered model 'Ridge' already exists. Creating a new version of this model...
2024/08/01 14:25:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Ridge, version 2


registered models: ['Lasso', 'Ridge'],           exp. name: Ridge


Created version '2' of model 'Ridge'.
